# Lógica Proposicional

Operações mais importantes:
* ```Atom(x)```
* ```Not(x)```
* ```And(x, y), AndList(x1,x2,...,xn)```
* ```Or(x, y)```, ```OrList(x1,x2,...,xn)```
* ```Xor(x,y)```
* ```Implies(x, y)```
* ```Equiv(x, y)```

Fluxo de utilização:
```python
kb = createResolutionKB()
kb.tell(<formula>) # add information to KB
kb.ask(<formula>)  # submit queries to KB
kb.dump()          # list derivations
```


In [3]:
from logic import *

Observe as derivações produzidas a partir das entradas exemplo, listadas por ```kb.dump()```.
* Fórmulas de entrada ```('*', <formula>)```
* Fórmulas derivadas ```('-', <formula>)```

In [3]:
pessoa, estudante = Atom('Pessoa'), Atom('Estudante')
criativa, matriculada = Atom('Criativa'), Atom('Matriculada')

kb = createResolutionKB()
kb.tell(Implies(estudante, pessoa))
kb.tell(Implies(estudante, criativa))
kb.tell(Equiv(matriculada, estudante))
kb.dump()

==== Knowledge base [6 derivations] ===
('*', Or(Not(Estudante),Pessoa))
('*', Or(Criativa,Not(Estudante)))
('*', Or(Estudante,Not(Matriculada)))
('-', Or(Not(Matriculada),Pessoa))
('-', Or(Criativa,Not(Matriculada)))
('*', Or(Matriculada,Not(Estudante)))


In [5]:
# para uma determinada instância, matriculada é verdadeiro
# observe as derivações produzidas
kb.tell(matriculada)
kb.dump()

==== Knowledge base [10 derivations] ===
('*', Or(Not(Estudante),Pessoa))
('*', Or(Criativa,Not(Estudante)))
('*', Or(Estudante,Not(Matriculada)))
('-', Or(Not(Matriculada),Pessoa))
('-', Or(Criativa,Not(Matriculada)))
('*', Or(Matriculada,Not(Estudante)))
('*', Matriculada)
('-', Estudante)
('-', Pessoa)
('-', Criativa)


## A visita de Sócrates a Platão

Se Platão estiver disposto a visitar Sócrates então Sócrates está disposto a visitar Platão. <br>
Se Sócrates estiver disposto a visitar Platão então Platão não está disposto a visitar Sócrates. <br>
Se Sócrates não estiver disposto a visitar Platão então Platão está disposto a visitar Sócrates. <br>

**Sócrates está disposto a visitar Platão?**

In [25]:
from logic import *

sp, ps = Atom('Socrates quer visitar Platao'), Atom('Platao quer visitar Socrates')

kb = createResolutionKB()
kb.tell(Implies(ps,sp))
kb.tell(Implies(sp,Not(ps)))
kb.tell(Implies(Not(sp),ps))
kb.ask(sp)

Yes.

## Mentirosos e Honestos

Considere que você está numa realidade onde as pessoas sempre mentem ou sempre dizem a verdade. Você encontra duas pessoas, Fulana e Beltrana. Fulana diz: “Pelo menos uma de nós é mentirosa.” 

**Fulana é a pessoa que sempre mente ou a que sempre diz a verdade? E Beltrana? Justifique.**

In [29]:
from logic import *

f, b = Atom('Fulana é mentirosa'), Atom('Beltrana é mentirosa')

kb = createResolutionKB()
kb.tell(Implies(f,Not(b)))
kb.tell(Implies(b,Not(f)))
kb.tell(Implies(Not(f),And(f,b)))

kb.ask(f)
kb.ask(b)
kb.dump()


==== Knowledge base [4 derivations] ===
('*', Or(Not(Beltrana é mentirosa),Not(Fulana é mentirosa)))
('*', Fulana é mentirosa)
('-', Not(Beltrana é mentirosa))
('*', Or(Beltrana é mentirosa,Fulana é mentirosa))


## Puzzle: Quem derrubou a Internet?

Alguém derramou água no roteador da Universidade e ficamos sem Internet! As pessoas, muito estressadas, começaram a jogar acusações. Sabendo que só uma pessoa pode ter feito isso, e que só uma das afirmações a seguir é verdade, descubra de quem foi a responsabilidade.

* Maria disse: "Não fui eu!"
* Gustavo disse: "Foi July."
* July disse: "Não! Foi Henrique."
* Henrique disse: "July está mentindo!"

In [46]:
students = ['Maria', 'Gustavo', 'July', 'Henrique']

# Só uma pessoa pode ter feito isso
worlds = []
for student in students:
    world = [Atom(s) if s == student else Not(Atom(s)) for s in students]
    print(world)
    worlds.append(world)
print()

# Só uma das afirmações é verdade
### Fórmulas
formulas = {'Maria': Not(Atom('Maria'))}     # Maria disse
formulas['Gustavo']  = Atom('July')          # Gustavo disse
formulas['July']     = And(Not(Atom('July')), Atom('Henrique')) # July disse
formulas['Henrique'] = Not(formulas['July']) # Henrique disse [ Nicole or not(Susan) ]
print(formulas) 

[Maria, Not(Gustavo), Not(July), Not(Henrique)]
[Not(Maria), Gustavo, Not(July), Not(Henrique)]
[Not(Maria), Not(Gustavo), July, Not(Henrique)]
[Not(Maria), Not(Gustavo), Not(July), Henrique]

{'Maria': Not(Maria), 'Gustavo': July, 'July': And(Not(July),Henrique), 'Henrique': Not(And(Not(July),Henrique))}


Se apenas uma pessoa está dizendo a verdade, então pela lógica temos 3 contradições. Teste todos os possíveis mundos, criando uma base de conhecimento para cada um deles (tell) e perguntando quais afirmações dos estudantes geram contradições (ask).  

Ao final responda: *Quem derrubou a Internet?*

In [41]:
for world in worlds:
    liarKB = createResolutionKB()
    [liarKB.tell(w) for w in world]
    liarKB.dump()
    
    for key in formulas.keys():
        print(f"{key}: {formulas[key]} - {liarKB.ask(formulas[key])}")
    print('--------------')

==== Knowledge base [4 derivations] ===
('*', Mark)
('*', Not(John))
('*', Not(Nicole))
('*', Not(Susan))
Mark: Not(Mark) - No.
John: Nicole - No.
Nicole: And(Not(Nicole),Susan) - No.
Susan: Not(And(Not(Nicole),Susan)) - Yes.
--------------
==== Knowledge base [4 derivations] ===
('*', Not(Mark))
('*', John)
('*', Not(Nicole))
('*', Not(Susan))
Mark: Not(Mark) - Yes.
John: Nicole - No.
Nicole: And(Not(Nicole),Susan) - No.
Susan: Not(And(Not(Nicole),Susan)) - Yes.
--------------
==== Knowledge base [4 derivations] ===
('*', Not(Mark))
('*', Not(John))
('*', Nicole)
('*', Not(Susan))
Mark: Not(Mark) - Yes.
John: Nicole - Yes.
Nicole: And(Not(Nicole),Susan) - No.
Susan: Not(And(Not(Nicole),Susan)) - Yes.
--------------
==== Knowledge base [4 derivations] ===
('*', Not(Mark))
('*', Not(John))
('*', Not(Nicole))
('*', Susan)
Mark: Not(Mark) - Yes.
John: Nicole - No.
Nicole: And(Not(Nicole),Susan) - Yes.
Susan: Not(And(Not(Nicole),Susan)) - No.
--------------
